In [2]:
from sklearn import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
dataset = datasets.load_breast_cancer()
X, y = dataset['data'], dataset['target']
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)
X = (X-torch.mean(X))/torch.std(X)
dset = TensorDataset(X, y)
loader = DataLoader(dset, batch_size=256, shuffle=True)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features, 4)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(4,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

In [6]:
model = NeuralNetwork(30)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [7]:
def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)


In [8]:
n_epochs = 100
for epoch in range(1, n_epochs+1):
    loss = train(model, criterion, optimizer, loader)
    if epoch % 10 == 0:
        print('epoch : {}, loss : {:.4f}'.format(epoch, loss))


epoch : 10, loss : 0.6784
epoch : 20, loss : 0.6658
epoch : 30, loss : 0.6511
epoch : 40, loss : 0.6509
epoch : 50, loss : 0.6598
epoch : 60, loss : 0.6764
epoch : 70, loss : 0.6817
epoch : 80, loss : 0.6573
epoch : 90, loss : 0.6598
epoch : 100, loss : 0.6430


In [11]:
y_predicted1 = (model(X)>=0.5).float()
score_of_trained_model = (y_predicted1==y).float().mean()
print('accuracy of trained model : {:.2f}'.format(score_of_trained_model))
torch.save(model.state_dict(), './trained_model.pt')

loaded_model = NeuralNetwork(30)
loaded_model.load_state_dict(torch.load('./trained_model.pt'))
y_predicted2 = (loaded_model(X)>=0.5).float()
score_of_loaded_model = (y_predicted2 == y).float().mean()
print('accuracy of loaded model : {:.2f}'.format(score_of_loaded_model))

accuracy of trained model : 0.63
accuracy of loaded model : 0.63
